# Settings

## Imports

In [1]:
import sys,tweepy,csv,re, requests, json
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import os, time, re


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gustavo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Configs

In [7]:
import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore') 

#Changing dir
folder = 'ML'
try:
    os.chdir(f'{os.getcwd()}/{folder}')
except:
    dir = os.getcwd().replace(os.getcwd().split('/')[-1], folder)
    os.chdir(dir)

# Getting Twitter

## API

In [11]:
#token
config = dotenv_values(".env")
bearer_token = config['BEARER_TOKEN']

#connections
auth = tweepy.OAuth2BearerHandler({bearer_token})
api = tweepy.API(auth)




def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def get_tweet_v1(query, filename, max_n):
    search_url = 'https://api.twitter.com/2/tweets/search/recent'


    file_name = f'{filename}.bz2'
    
    if os.path.exists(file_name) == False: #First checking if database exists
        print(f'Getting tweets...')
        
        # Querying the API
        json_response = connect_to_endpoint(search_url, query)
        
        tweets_dt = pd.DataFrame.from_dict(json_response['data'])
        
        try:
            n_token = json_response['meta']["next_token"]
            n = 0
            while n_token != 0 | n < max_n:
                print(f'Next Token: {n} \n {n_token}')
                query_next = query
                query_next['next_token'] = n_token
                json_response = connect_to_endpoint(search_url, query_next)
                tweets_n = pd.DataFrame.from_dict(json_response['data'])
                tweets_dt = pd.concat([tweets_dt,tweets_n], ignore_index=True)

                n += 1
                n_token = json_response['meta']["next_token"]

        except:
            print('Error to proceed')
            
        meta = json_response['meta']
        #np.save(f'{filename}.npy', meta)
        #print ('file Meta Saved')
 
        #tweets_dt.to_csv(file_name, index=False,compression='bz2')
        #print(f'{len(tweets_dt)} Tweets found and saved')
        
    else:
        create_dt = time.strftime("%d/%m/%Y %H:%M:%S",time.strptime(time.ctime(os.path.getmtime(file_name))))
        print(f'Reading {file_name}, created at {create_dt}')
        tweets_dt = pd.read_csv(file_name)
        print(f'File with {len(tweets_dt)} Tweets')
        
    return tweets_dt


## Countries

Getting up to 1000 tweets each country

In [12]:
countries = ['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Denmark', 'Finland', 'France', 'Germany', 'Hungary',
        'Ireland', 'Italy', 'Latvia', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Spain','Sweden', 'Europe', 'UK']

filename = 'tweets_agri_2.bz2'

if os.path.exists(filename) == False: 
    for c in countries:

        q = {
        'query': f'agriculture {c} -is:retweet',
        'max_results': 100,
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        }

        fn = f'tweets_{c}'
        t = get_tweet_v1(q, fn, 10)
        t['country'] = c
        if c == countries[0]:
            tweets = t
        else:
            tweets = pd.concat([tweets, t], ignore_index=True)

    tweets.to_csv(filename, index=False,compression='bz2')
    print(f'File with {len(tweets)} saved.')

else:
    create_dt = time.strftime("%d/%m/%Y %H:%M:%S",time.strptime(time.ctime(os.path.getmtime(filename))))
    print(f'Reading {filename}, created at {create_dt}')
    tweets = pd.read_csv(filename)
    print(f'File with {len(tweets)} Tweets')
    

tweets.sample(3)

Getting tweets...
Error to proceed
file Meta Saved
2 Tweets found and saved
Getting tweets...
Error to proceed
file Meta Saved
3 Tweets found and saved
Getting tweets...
Error to proceed
file Meta Saved
1 Tweets found and saved
Getting tweets...


KeyError: 'data'

# Clean Your Text Data

## Worlds extractions

In [43]:
def clean_tweet(x, link, keyword, usernames):
    list_of_lists =[]
    if link == True:
        list_of_links = []
        words = x.split(' ')
        for word in words:
            if re.search('http', word):
                list_of_links.append(re.split("\W+",word.lower()))
        if len(list_of_links) > 0:
            list_of_lists.append(list_of_links[0])
    
    if keyword == True:
        list_of_keywords = []
        words = x.split()
        for word in words:
            if word.startswith('#'):
                list_of_keywords.append(word)
        if len(list_of_keywords) > 0:
            list_of_lists.append(list_of_keywords)
            
    if usernames == True:
        list_of_usernames = []
        words = x.split()
        for word in words:
            if word.startswith('@'):
                list_of_usernames.append(word.lower().replace('@',''))
        if len(list_of_usernames) > 0:
            list_of_lists.append(list_of_usernames)
    
    return  [item for sublist in list_of_lists for item in sublist]

        
# keyword extraction from tweets
def get_keywords(x):
    list_of_keywords = []
    words = x.split()
    for word in words:
        if word.startswith('#'):
            list_of_keywords.append(word)
    return list_of_keywords


In [44]:
tweets = tweets[tweets.lang == 'en']
tweets = tweets.reset_index(drop = True)

In [45]:
list_of_lists = tweets['text'].apply(lambda tweet : clean_tweet(tweet, link = True, keyword = False, usernames = True))
rem_list = [item for sublist in list_of_lists for item in sublist]

tweets['text_c'] = tweets['text'].apply( lambda tweet : ' '.join([word for word in re.split("\W+",tweet) if word.lower() not in rem_list]))

In [46]:
tweets['keywords'] = tweets['text'].apply( lambda tweet : get_keywords(tweet) )

In [47]:
tweets.sample(2)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords
174,The headquarter of Food and Agriculture Organi...,1606197452314251266,en,everyone,1606197452314251266,1258905310699425792,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1606197452314251266'],2022-12-23T07:58:08.000Z,NaN,NaN,NaN,France,headquarter of Food Organization of United Nat...,[#READ_MORE]
241,"Before we all start our holidays, we are pleas...",1605502032919506944,en,everyone,1605502032919506944,34932200,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605502032919506944'],2022-12-21T09:54:48.000Z,NaN,NaN,NaN,German,Before we all start our holidays we pleased to...,[]


In [48]:
n = 1
print('OLD: ', tweets['text'][n], '\n')
print('NEW: ', tweets['text_c'][n])

OLD:  @Nien72521217 @divyanshu3pathi The per Capita story doesn't correlate

When I see 1000 AD, W Europe didn't have trade routes established, didn't have industrial revolution. Had largely constrained agriculture due to nature of their land. No known irrigation projects.

Yet the per Capita is high? 

NEW:  per Capita story doesn correlate When I see 1000 AD W Europe didn have routes established didn have industrial revolution Had largely constrained due to nature of their land No known irrigation projects Yet per Capita is high


## PoterStemmer

In [49]:
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()

In [50]:
def stremming(df, text_col, name_new_col):
    # Store the column of the dataframe named as "text"
    X = df[text_col]
    cleaned_data=[]
    # For loop from first value to length(X), ^a-zA-Z means include small and capital case letters
    for i in range(len(X)):
        text = re.sub('[^a-zA-Z]', ' ', X.iloc[i])
        text = text.lower().split()
        text = [stemmer.stem(word) for word in text if (word not in stop_words) and (word not in punct)]
        text = ' '.join(text)
        df.loc[ i ,name_new_col] = text
    print('Stremmer done!')

In [51]:
stremming(tweets, 'text_c', 'text_ps')

Stremmer done!


In [52]:
print(tweets['text'][0], '\n')
print(tweets['text_ps'][0], '\n')

The knowledge of how to make pottery spread like wildfire between the hunter-gatherers of Stone Age Europe, covering thousands of miles in a few hundred years, much faster than agriculture https://t.co/Iu2YACvIGN 

knowledg make potteri spread like wildfir hunter gather stone age europ cover thousand mile hundr much faster 



## Sentiment Analyzes

TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more. [link](https://textblob.readthedocs.io/en/dev/index.html)

In [53]:
tweets.sample(2)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords,text_ps
336,@TAH_Sci You are speaking to a sneaky reader o...,1605869709844439040,en,everyone,1605886458937720836,1578305394,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605886458937720836'],2022-12-22T11:22:22.000Z,816003098.0,"[{'type': 'replied_to', 'id': '160587271910641...",NaN,UK,You speaking to a sneaky reader of Farmers Gua...,[],speak sneaki reader farmer guardian farmer wee...
153,Thank you to Minister @MartinHeydonFG for atte...,1604806390282862599,en,everyone,1604806390282862599,1235536533077270530,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",['1604806390282862599'],2022-12-19T11:50:33.000Z,NaN,NaN,{'place_id': 'cbd95de037c865b4'},Ireland,Thank you to Minister for attending demonstrat...,[],thank minist attend demonstr member agri guard...


In [54]:
for i in tweets.index:
    text = tweets.loc[i,'text_ps']
    tweets.loc[i, 'TextBlob'] = TextBlob(text).sentiment.polarity
    #print(TextBlob(text).sentiment.polarity)
    tweets.loc[i, 'Vader'] = SentimentIntensityAnalyzer().polarity_scores(text)['compound']
    #print(SentimentIntensityAnalyzer().polarity_scores(text)['compound'], '\n')
    
tweets.sample(2)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords,text_ps,TextBlob,Vader
388,"Despite the war, the #wheat harvest 🌾 in #Ukra...",1605568824371191809,en,everyone,1605568824371191809,1090218748282310657,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605568824371191809'],2022-12-21T14:20:12.000Z,NaN,NaN,NaN,UK,Despite wheat harvest in has been larger than ...,"[#wheat, #Ukraine]",despit wheat harvest larger expect tell,0.000000,0.0000
68,What do farmers think of the #NewCAP?\n\n🎧 In ...,1605225822683820033,en,everyone,1605225822683820033,2885085044,"{'retweet_count': 7, 'reply_count': 0, 'like_c...",['1605225822683820033'],2022-12-20T15:37:14.000Z,NaN,NaN,NaN,Europe,What farmers think of NewCAP In new episode of...,"[#NewCAP?, #Food4EU]",farmer think newcap new episod food europ podc...,0.136364,-0.2732


In [55]:
def get_polarity(df, col):

    polarity = 0
    neutral = 0
    wpositive = 0
    positive = 0
    spositive = 0
    wnegative = 0
    negative = 0
    snegative = 0
     
    for t in df.index:
        
        v = df.loc[t, col]
        polarity += v  # adding up polarities to find the average later

        if (v == 0):  # adding reaction of how people are reacting to find average later
            neutral += 1
            desc = 'neutral'
        elif (v > 0 and v <= 0.3):
            wpositive += 1
            desc ='weak_positive'
        elif (v > 0.3 and v <= 0.6):
            positive += 1
            desc = 'positive'
        elif (v > 0.6 and v <= 1):
            spositive += 1
            desc = 'strong_positive'
        elif (v > -0.3 and v <= 0):
            wnegative += 1
            desc = 'weak_negative'
        elif (v > -0.6 and v <= -0.3):
            negative += 1
            desc = 'negative'
        elif (v > -1 and v <= -0.6):
            snegative += 1
            desc = 'strong_negative'
         
        df.loc[t, f'{col}_desc'] = desc
        

    return {'polarity_sum':polarity,
            'polarity_mean':(polarity / len(df)),
            'neutral':neutral,
            'strong_positive':spositive,
            'positive':positive,
            'weak_positive':wpositive,
            'weak_negative':wnegative,
            'negative':negative,
            'strong_negative':snegative}




In [56]:
get_polarity(tweets,'TextBlob')

{'polarity_sum': 43.62266860916862,
 'polarity_mean': 0.06837408872910442,
 'neutral': 265,
 'strong_positive': 18,
 'positive': 68,
 'weak_positive': 174,
 'weak_negative': 91,
 'negative': 13,
 'strong_negative': 8}

In [57]:
get_polarity(tweets,'Vader')

{'polarity_sum': 109.63159999999992,
 'polarity_mean': 0.1718363636363635,
 'neutral': 209,
 'strong_positive': 99,
 'positive': 154,
 'weak_positive': 70,
 'weak_negative': 39,
 'negative': 42,
 'strong_negative': 25}

In [58]:
tweets.sample()

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,referenced_tweets,geo,country,text_c,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc
470,"The Department of Agriculture, Environment and...",1605172950457618432,en,everyone,1605172950457618432,4849821129,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605172950457618432'],2022-12-20T12:07:08.000Z,NaN,NaN,NaN,UK,Department of Rural Affairs DAERA has announce...,[],depart rural affair daera announc public data ...,0.0,0.0,neutral,neutral


In [59]:
##Checking Twitters

n = 3
print('Text Original:', tweets.loc[n, 'text'], '\n',
     '-------------------------------------------------------------------------------------------')
print('Text Clear:', tweets.loc[n, 'text_c'], '\n',
     '-------------------------------------------------------------------------------------------')

print('Text Steammed:', tweets.loc[n, 'text_ps'], '\n',
     '-------------------------------------------------------------------------------------------')

print('KeyWords:', tweets.loc[n, 'keywords'], '\n',
     '-------------------------------------------------------------------------------------------')

print('TextBlob: ',tweets.loc[n, 'TextBlob'], tweets.loc[n, 'TextBlob_desc'])
print('Vader: ', tweets.loc[n, 'Vader'], tweets.loc[n, 'Vader_desc'])

Text Original: @RonFilipkowski Ron, you are missing the point.  Putin doesn’t want peace, Putin wants Ukraine’s oil, gas and agriculture (wheat) to have a stranglehold on Europe!!! 
 -------------------------------------------------------------------------------------------
Text Clear: Ron you missing point Putin doesn want peace Putin wants s oil gas wheat to have a stranglehold on Europe 
 -------------------------------------------------------------------------------------------
Text Steammed: ron miss point putin want peac putin want oil ga wheat stranglehold europ 
 -------------------------------------------------------------------------------------------
KeyWords: [] 
 -------------------------------------------------------------------------------------------
TextBlob:  0.0 neutral
Vader:  0.0 neutral


In [60]:
tweets.iloc[:, -6:]

,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc
0,[],knowledg make potteri spread like wildfir hunt...,0.200000,0.3612,weak_positive,positive
1,[],per capita stori correl see ad w europ rout es...,0.017500,0.0000,weak_positive,neutral
2,[],everyth need chang villag settl illo tempor hu...,0.000000,0.7717,neutral,strong_positive
3,[],ron miss point putin want peac putin want oil ...,0.000000,0.0000,neutral,neutral
4,[],russia one encourag us aid amp assist help peo...,0.000000,0.4019,neutral,positive
...,...,...,...,...,...,...
633,[],ye reform set safe amp legal rout process refu...,0.350000,0.8689,positive,strong_positive
634,"[#BiggestJobonEarth, #agriculture]",case miss catch episod biggestjobonearth podca...,-0.500000,0.1531,negative,weak_positive
635,[],respons contribut transform new strategi,0.136364,0.0000,weak_positive,neutral
636,[],eu minist call rethink free,0.400000,0.5106,positive,positive


## Sarcasm

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
news = pd.read_csv('sarcasm_headlines.bz2')
news.sample()

,headline,is_sarcastic
22179,depressed nra member half-hoping son will acci...,1


In [63]:
print(news.isnull().any(axis = 0))

headline        False
is_sarcastic    False
dtype: bool


In [64]:
stremming(news, 'headline', 'headline_ps')

Stremmer done!


In [65]:
news.sample(2)

,headline,is_sarcastic,headline_ps
5178,report: spider,1,report spider
22605,blood-sucking lamprey forced to make awkward s...,1,blood suck lamprey forc make awkward small tal...


In [66]:
def create_features(df, col, max_feat):
    features = df[col]

    # vectorizing the data with maximum features
    tv = TfidfVectorizer(max_features = max_feat)
    features = list(features)
    features = tv.fit_transform(features).toarray()
    
    return features

In [67]:
features = create_features(news, 'headline_ps', 3300)
labels = news['is_sarcastic']

In [68]:
# getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

In [69]:
print('\nLinear Support Vector Classifier:')
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print('Train: ',lsvc.score(features_train, labels_train))
print('Test: ',lsvc.score(features_test, labels_test))


print('\nGaussuan Naive Bayes:')
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('Train: ',gnb.score(features_train, labels_train))
print('Test: ',gnb.score(features_test, labels_test))


print('\nLogistic Regression:')
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('Train: ',lr.score(features_train, labels_train))
print('Test: ',lr.score(features_test, labels_test))


print('\nRandom Forest Classifier:')
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('Train: ',rfc.score(features_train, labels_train))
print('Test: ',rfc.score(features_test, labels_test))


Linear Support Vector Classifier:
Train:  0.8462933039057265
Test:  0.7784431137724551

Gaussuan Naive Bayes:
Train:  0.750679856540417
Test:  0.7215568862275449

Logistic Regression:
Train:  0.8314349899499468
Test:  0.782185628742515

Random Forest Classifier:
Train:  0.9827375556694123
Test:  0.7365269461077845


### Prev

In [70]:
prev = create_features(tweets, 'text_ps', 3300)

In [71]:
tweets['is_sarcastic_lsvc'] = lsvc.predict(prev)

In [72]:
tweets['is_sarcastic_lr'] = lr.predict(prev)

In [73]:
tweets.sample(4)

,text,conversation_id,lang,reply_settings,id,author_id,public_metrics,edit_history_tweet_ids,created_at,in_reply_to_user_id,...,country,text_c,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc,is_sarcastic_lsvc,is_sarcastic_lr
258,@GermanyDiplo @DrSJaishankar @ABaerbock Cpec r...,1599737279890661377,en,everyone,1604820915136696322,1566407156576002050,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1604820915136696322'],2022-12-19T12:48:16.000Z,453030125.0,...,German,Cpec rpec projects electric gas dam sugar stee...,[],cpec rpec project electr ga dam sugar steel mi...,0.077778,0.4404,weak_positive,positive,0,0
55,The future of food - Is sustainable agricultur...,1605529594471161857,en,everyone,1605529594471161857,1099450872784797696,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605529594471161857'],2022-12-21T11:44:19.000Z,NaN,...,Europe,future of food Is sustainable possible in Euro...,[],futur food sustain possibl europ via,0.000000,0.0000,neutral,neutral,0,0
223,#EuropeanUnion #France\nFlashback on ECJ Cases...,1603993929959067653,en,everyone,1603993929959067653,1035079865412845568,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",['1603993929959067653'],2022-12-17T06:02:08.000Z,NaN,...,France,EuropeanUnion France Flashback on Cases C 596 ...,"[#EuropeanUnion, #France, #ECJ(EuropeanCourtof...",europeanunion franc flashback case c commiss v...,0.000000,0.0000,neutral,neutral,0,0
335,"New blog 📣\n\nAccording to a recent study, #fa...",1605891783866597376,en,everyone,1605891783866597376,280867595,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",['1605891783866597376'],2022-12-22T11:43:31.000Z,NaN,...,UK,New blog According to a recent study farmers m...,"[#farmers, #Agriculture]",new blog accord recent studi farmer make tini ...,-0.058712,0.4404,weak_negative,positive,1,1


In [74]:
##Checking Twitters

n = 101
print('Text Original:', tweets.loc[n, 'text'], '\n',
     '-------------------------------------------------------------------------------------------')
print('Text Clear:', tweets.loc[n, 'text_c'], '\n',
     '-------------------------------------------------------------------------------------------')

print('Text Steammed:', tweets.loc[n, 'text_ps'], '\n',
     '-------------------------------------------------------------------------------------------')

print('KeyWords:', tweets.loc[n, 'keywords'], '\n',
     '-------------------------------------------------------------------------------------------')

print('TextBlob: ',tweets.loc[n, 'TextBlob'], tweets.loc[n, 'TextBlob_desc'])
print('Vader: ', tweets.loc[n, 'Vader'], tweets.loc[n, 'Vader_desc'])
print('Sarcasm (lsvc): ', tweets.loc[n, 'is_sarcastic_lsvc'])
print('Sarcasm (lr): ', tweets.loc[n, 'is_sarcastic_lr'])


Text Original: @andersen_inger @UrosBrezan @SLOtoUN @UNEP_Europe @rs_mop @MOEPPMKD I have solutions to increase water sources reducing atmosphere temperature with 15C stopping glaciers melting, eliminating smog of Santiago, to prevent forest fires, to make grassland, agriculture land, forest, în slop,desert, mechanized, efficient, cheap Please send answer Thank 
 -------------------------------------------------------------------------------------------
Text Clear: I have solutions to increase water sources reducing atmosphere temperature with 15C stopping glaciers melting eliminating smog of Santiago to prevent forest fires to make grassland land forest în slop desert mechanized efficient cheap Please send answer Thank 
 -------------------------------------------------------------------------------------------
Text Steammed: solut increas water sourc reduc atmospher temperatur c stop glacier melt elimin smog santiago prevent forest fire make grassland land forest n slop desert mechan